# Applied Digital Citizen Science
## Session 8

DISCLOSURE: Parts of the code for this notebook were created with GitHub CoPilot, and disclosed as such. All code has been tested by the lecturers.

See Canvas for details about usage of Generative AI in assignments

## Objectives for Session 8

The code here will be relevant for cleaning, aggregating and formatting the data that you will use for your written report. 

The sections of code in this notebook are self-contained, and you may not need to use everything all the time. The objectives of this session are:

1. Combining the raw dataset (generated with the code from session 6) with self-reports
2. Combining the raw dataset (generated with the code from session 6) with the YouTube Data Tools
3. Aggregating the dataset at the correct level depending on your RQ

## 1. Combining the donated dataset with self-reports

For these steps, you first need to:
1. Run the steps in Session 6 that generate the consolidated datasets (for example on watch or search)
2. Download the data from Qualtrics as Excel
3. Make sure all datasets are in the same folder as the script you are running

The example below uses the watch history. You can simply load another dataset (e.g., search) and run the same steps.

In [ ]:
import pandas as pd

In [ ]:
donated_data = pd.read_excel('Watch.xlsx')

In [ ]:
donated_data

In [ ]:
self_reports = pd.read_excel('ADCS-2025-demo2_September+21,+2025_17.03.xlsx', skiprows=[1])

In [ ]:
self_reports

In [ ]:
def extract_participant_id(text):
    text = text.split('_')
    if len(text) > 2:
        return text[2]
    return None

In [ ]:
self_reports['participantid'] = self_reports['participant'].apply(extract_participant_id)

In [ ]:
self_reports

In [ ]:
donated_data['participantid'].value_counts()

In [ ]:
self_reports['participantid'].value_counts()

### Options for merging the data

- We could merge the data at the video level (i.e., each participant appears 20 times in the dataset)
- We could merge the data at participant level (i.e., each participant appears once in the dataset)

Which way to choose? Depends on the RQ.

For now, we will have examples in both ways - both at the video level, and at the participant level.

### Merging the data at the video level

In [ ]:
dataset_video_level = pd.merge(donated_data, self_reports, on='participantid', how='left')

In [ ]:
dataset_video_level

In [ ]:
dataset_video_level['participantid'].value_counts()

In [ ]:
self_reports['participantid'].value_counts()

For discussion: Do we have the same participants in both? If not, why?

### Merging at the participant level

For this dataset, each participant will appear only once. Considering that in the videos dataset each participant appears multiple times, we will then need to decide how do we aggregate the videos dataset at participant level.

In this example, I will use the number of videos watched.

In [ ]:
videos_watched = pd.pivot_table(donated_data, index='participantid', values='Link', aggfunc='count').reset_index()

In [ ]:
videos_watched

For discussion: Why do I only have 19 videos, if each participant appears 20 times in the donated dataset?

In [ ]:
dataset_participant_level = pd.merge(self_reports, videos_watched, on='participantid', how='left').rename(columns={'Link': 'videos_watched'})

In [ ]:
dataset_participant_level

In [ ]:
dataset_participant_level.isna().sum()

In [ ]:
dataset_participant_level['videos_watched'].describe()

For discussion: Why do some participants have a missing value in videos watched?

### Exporting the datasets

After completing the work, you may want to export the appropriate datasets. The example below has the dataset_participant_level as an example.

In [ ]:
dataset_participant_level.to_excel('dataset_participant_level.xlsx', index=False)

## 2. Combining the donated dataset with YouTube Data Tools

For these steps, you first need to:
1. Run the steps in Session 6 that generate the consolidated datasets (for example on watch or search)
2. Make sure all datasets are in the same folder as the script you are running

The example below uses the watch history. 

In [1]:
import pandas as pd

In [2]:
watch_history = pd.read_excel('Watch.xlsx')

In [3]:
watch_history

,Unnamed: 0,Titel,Link,Datum en tijd,group,participantid
0,0,Watched AW25-Programmatic-273_Youtube-Masthead...,https://www.youtube.com/watch?v=-CIz8miEGrI,2025-09-28T09:49:20.771Z,group02,13
1,1,Watched screencasts gmt94 3v en Check Rewards ...,https://www.youtube.com/watch?v=bb8QFm-5z_M,2025-09-19T20:24:11.047Z,group02,13
2,2,Viewed Ads On YouTube Homepage,NaN,2025-09-18T20:49:57.036Z,group02,13
3,3,Watched 0923-b6-h50A,https://www.youtube.com/watch?v=SkcwLeP-KZw,2025-09-18T20:22:18.937Z,group02,13
4,4,Watched 🔴 LIVE - USS Des Moines | World of War...,https://www.youtube.com/watch?v=tV9_5ruXS88,2025-09-18T20:01:22.877Z,group02,13
...,...,...,...,...,...,...
1720,1720,Watched The Summer I Turned Pretty Season 3 - ...,https://www.youtube.com/watch?v=UFPT7_U4oyc,2025-09-04T09:35:42.147Z,group02,4
1721,1721,Watched RITUALS DELUXE ADVENTSKALENDER UNBOXIN...,https://www.youtube.com/watch?v=IuO5K4ZVxrc,2025-09-04T09:35:26.082Z,group02,4
1722,1722,Watched Douglas Exclusive Beauty Highlights 20...,https://www.youtube.com/watch?v=VBkpWfWOYQc,2025-09-04T09:35:14.188Z,group02,4
1723,1723,Watched HOURGLASS AMBIENT LIGHTING EDIT UNLOCK...,https://www.youtube.com/watch?v=2vzgW6sOS-I,2025-09-04T09:32:26.458Z,group02,4


We now need to get the video_ids from the YouTube videos. Remember the code we saw on Session 4. 

(Disclosure: the code below was created with the help of CoPilot, and was updated for this session).

In [4]:
def extract_youtube_video_id(url):
    url = str(url)
    if 'youtube.com/watch?v=' in url:
        return url.split('v=')[1].split('&')[0]
    elif 'youtu.be/' in url:
        return url.split('youtu.be/')[1].split('?')[0]
    else:
        return None


In [5]:
watch_history['videoid'] = watch_history['Link'].apply(extract_youtube_video_id)

In [6]:
watch_history['videoid']

0       -CIz8miEGrI
1       bb8QFm-5z_M
2              None
3       SkcwLeP-KZw
4       tV9_5ruXS88
           ...     
1720    UFPT7_U4oyc
1721    IuO5K4ZVxrc
1722    VBkpWfWOYQc
1723    2vzgW6sOS-I
1724    RJwRThfQNI4
Name: videoid, Length: 1725, dtype: object

Now I will print the video ids in a format that I can simply then copy and paste to reuse at the YouTube Data Tools interface.

In [7]:
for videoid in watch_history['videoid'].unique():
    if len(str(videoid)) > 4:
        print(videoid + ',')

-CIz8miEGrI,
bb8QFm-5z_M,
SkcwLeP-KZw,
tV9_5ruXS88,
vSR3G2_so3A,
T3pCPOUUflA,
JA74jfDqYuo,
AmW5YuJRO6M,
NRBb0hEF63A,
Y9keSa7CUx4,
1uEnu6POSGY,
rKlrFVbnUl0,
eBXjxg_TdLM,
L7vZwJgcwYk,
Q5jkfUWdqYs,
tv66kzzSLMA,
N__TyEBA5YE,
OboCDDMAxxQ,
PSmWsO9gCQQ,
d3YuDqi5bQk,
OugxDOargKI,
mPOsHpEQaLA,
G8NsrmKHXbc,
f82GHUz2C80,
och_AU39Zgo,
mdcE0Qfoddk,
rnBtoyqcgek,
Ul_QVZ3c7NI,
5QU4pjZpdyM,
VHAK-gU9gi0,
VQRLujxTm3c,
KcERdzAiC3s,
vmVz5fyPgdc,
n2g2I0F8DUQ,
mdr6rAB_OlI,
yL3vF17TOK8,
B-2haa01CMU,
jt7r7C6khv4,
n-ZKmjb7-qs,
FEjDqZAYR7M,
bU_nfmAIdf4,
MMinzfpS9I0,
nLLWGF-OSvE,
ydxQxYHuPM0,
11R12oFEhxA,
wnuknpizDnw,
61lr1wsU89M,
HuR3nqxvTDc,
xg83bQUYXuE,
vF43b_ptzzk,
6ixot7MKC54,
MIhtwUJC22w,
dLs7UusnhLc,
GicObjmOSRA,
z5RqkDBtUHw,
h5ZalF4UjzE,
vM3-sTLW_G8,
tRWqsXK2UfI,
D3oO_9S6TVM,
FaIudlj5Izg,
OmdBYTZ2Nwo,
ivniN_0rNas,
ouBa5h7fuC4,
PWTnZvJuF_A,
bSwT1Smh5z4,
9MYXEf-kg_I,
nOxjL_LE5f4,
uisK4sWVOnI,
9dv2ElQ-CAA,
2kKlzgLD4kA,
i39H53bisQk,
s7uBjmGK2UM,
n8nQKRkT7cw,
t_Glc1pG3uk,
epF7tTspo6o,
UIXPw369Udg,
yvoJMOx8L9E,

I will now go to the YouTube Data Tools and use these ids to generate a report with information about each of these videos (using the video list module). After this is done, I will download the report from YouTube Data Tools to my own computer, and save the file to the same folder as where this script and other relevant files are located.

In [8]:
youtube_report = pd.read_csv('videolist_seeds1393_2025_10_08-11_55_06.csv')

In [9]:
youtube_report

,position,channelId,channelTitle,videoId,publishedAt,publishedAtSQL,videoTitle,videoDescription,tags,videoCategoryId,...,thumbnail_maxres,licensedContent,locationDescription,latitude,longitude,viewCount,likeCount,dislikeCount,favoriteCount,commentCount
0,1,UCH8Kxr6Z5SQ8mj8HiF5syLw,Zalando,-CIz8miEGrI,2025-08-18T15:54:35Z,2025-08-18 15:54:35,AW25-Programmatic-273_Youtube-Masthead_a_Autum...,default description,NaN,26.0,...,https://i.ytimg.com/vi/-CIz8miEGrI/maxresdefau...,NaN,NaN,NaN,NaN,115628.0,21.0,NaN,0.0,2.0
1,2,UCQeWJrKN_ec5aoNwcLyiS8Q,GoMining,bb8QFm-5z_M,2025-08-20T08:28:21Z,2025-08-20 08:28:21,screencasts gmt94 3v en Check Rewards 16х9,NaN,NaN,28.0,...,https://i.ytimg.com/vi/bb8QFm-5z_M/maxresdefau...,NaN,NaN,NaN,NaN,9711747.0,29.0,NaN,0.0,2.0
2,3,UCBUsvFMskC0yuMYGSmqRO1w,Tom Crosshill,SkcwLeP-KZw,2024-01-25T13:21:02Z,2024-01-25 13:21:02,0923-b6-h50A,NaN,NaN,27.0,...,https://i.ytimg.com/vi/SkcwLeP-KZw/maxresdefau...,NaN,NaN,NaN,NaN,1264510.0,NaN,NaN,0.0,NaN
3,4,UCP8RshniIRZPw5HHeEqeh4w,WORLD OF WARSHIPS,tV9_5ruXS88,2022-07-06T15:41:49Z,2022-07-06 15:41:49,🔴 LIVE - USS Des Moines | World of Warships,🐱‍👤 Download the game: https://wo.ws/3zcUwu1 ...,"wows,world of warships,warships game",20.0,...,https://i.ytimg.com/vi/tV9_5ruXS88/maxresdefau...,NaN,NaN,NaN,NaN,76114020.0,NaN,NaN,0.0,18.0
4,5,UCA9UvBiKHly15rN8u_Km3BQ,monday.com,vSR3G2_so3A,2025-01-21T08:20:18Z,2025-01-21 08:20:18,Your team deserves a work platform they'll act...,monday.com helps your team love what they do b...,"management,platform,tool,teams,organization,mo...",28.0,...,https://i.ytimg.com/vi/vSR3G2_so3A/maxresdefau...,NaN,NaN,NaN,NaN,3674967.0,25.0,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1388,1389,UC-315YD2-S7gz_utpG52_eg,Make Me Blush,IuO5K4ZVxrc,2025-09-02T19:16:43Z,2025-09-02 19:16:43,RITUALS DELUXE ADVENTSKALENDER UNBOXING 2025 🩵...,↓MEER INFO↓ Shop de Rituals Classic Adventskal...,"makemeblush,make,me,blush,danielle,springer,be...",22.0,...,https://i.ytimg.com/vi/IuO5K4ZVxrc/maxresdefau...,1.0,NaN,NaN,NaN,23446.0,783.0,NaN,0.0,150.0
1389,1390,UCnYwJm9kVAs2p-88g1UjnCA,Helga Mua,VBkpWfWOYQc,2025-08-15T16:00:52Z,2025-08-15 16:00:52,Douglas Exclusive Beauty Highlights 2025. Adve...,Hey guys! Time to open a new Advent Calendar 2...,"helga mua,helga mua unboxing,advent calendar,a...",22.0,...,https://i.ytimg.com/vi/VBkpWfWOYQc/maxresdefau...,1.0,Munich,48.135125,11.581981,5419.0,147.0,NaN,0.0,41.0
1390,1391,UChK6kpyOTrMOJ391JUYBfSw,Charlotte Holdcroft,2vzgW6sOS-I,2025-09-03T13:04:16Z,2025-09-03 13:04:16,HOURGLASS AMBIENT LIGHTING EDIT UNLOCKED COLLE...,"Hi lovelies, Today I'm sharing all of my thoug...","hourglass ambient lighting palette 2025,hourgl...",26.0,...,https://i.ytimg.com/vi/2vzgW6sOS-I/maxresdefau...,1.0,NaN,NaN,NaN,27466.0,2389.0,NaN,0.0,326.0
1391,1392,UCfmqLyr1PI3_zbwppHNEzuQ,Grammarly,RJwRThfQNI4,2024-09-30T16:26:49Z,2024-09-30 16:26:49,Blow Your Workload Out of the Water | Find You...,It's 9:15am on a Monday. How are you already u...,NaN,27.0,...,https://i.ytimg.com/vi/RJwRThfQNI4/maxresdefau...,NaN,NaN,NaN,NaN,64638334.0,2225.0,NaN,0.0,NaN


Merging both datasets

In [10]:
watch_history_details = pd.merge(watch_history, youtube_report, left_on='videoid', right_on='videoId', how='left')

In [11]:
watch_history_details

,Unnamed: 0,Titel,Link,Datum en tijd,group,participantid,videoid,position,channelId,channelTitle,...,thumbnail_maxres,licensedContent,locationDescription,latitude,longitude,viewCount,likeCount,dislikeCount,favoriteCount,commentCount
0,0,Watched AW25-Programmatic-273_Youtube-Masthead...,https://www.youtube.com/watch?v=-CIz8miEGrI,2025-09-28T09:49:20.771Z,group02,13,-CIz8miEGrI,1,UCH8Kxr6Z5SQ8mj8HiF5syLw,Zalando,...,https://i.ytimg.com/vi/-CIz8miEGrI/maxresdefau...,NaN,NaN,NaN,NaN,115628.0,21.0,NaN,0.0,2.0
1,1,Watched screencasts gmt94 3v en Check Rewards ...,https://www.youtube.com/watch?v=bb8QFm-5z_M,2025-09-19T20:24:11.047Z,group02,13,bb8QFm-5z_M,2,UCQeWJrKN_ec5aoNwcLyiS8Q,GoMining,...,https://i.ytimg.com/vi/bb8QFm-5z_M/maxresdefau...,NaN,NaN,NaN,NaN,9711747.0,29.0,NaN,0.0,2.0
2,2,Viewed Ads On YouTube Homepage,NaN,2025-09-18T20:49:57.036Z,group02,13,None,1393,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Watched 0923-b6-h50A,https://www.youtube.com/watch?v=SkcwLeP-KZw,2025-09-18T20:22:18.937Z,group02,13,SkcwLeP-KZw,3,UCBUsvFMskC0yuMYGSmqRO1w,Tom Crosshill,...,https://i.ytimg.com/vi/SkcwLeP-KZw/maxresdefau...,NaN,NaN,NaN,NaN,1264510.0,NaN,NaN,0.0,NaN
4,4,Watched 🔴 LIVE - USS Des Moines | World of War...,https://www.youtube.com/watch?v=tV9_5ruXS88,2025-09-18T20:01:22.877Z,group02,13,tV9_5ruXS88,4,UCP8RshniIRZPw5HHeEqeh4w,WORLD OF WARSHIPS,...,https://i.ytimg.com/vi/tV9_5ruXS88/maxresdefau...,NaN,NaN,NaN,NaN,76114020.0,NaN,NaN,0.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720,1720,Watched The Summer I Turned Pretty Season 3 - ...,https://www.youtube.com/watch?v=UFPT7_U4oyc,2025-09-04T09:35:42.147Z,group02,4,UFPT7_U4oyc,1388,UCQJWtTnAHhEG5w4uN0udnUQ,Prime Video,...,https://i.ytimg.com/vi/UFPT7_U4oyc/maxresdefau...,1.0,NaN,NaN,NaN,3071058.0,51073.0,NaN,0.0,1968.0
1721,1721,Watched RITUALS DELUXE ADVENTSKALENDER UNBOXIN...,https://www.youtube.com/watch?v=IuO5K4ZVxrc,2025-09-04T09:35:26.082Z,group02,4,IuO5K4ZVxrc,1389,UC-315YD2-S7gz_utpG52_eg,Make Me Blush,...,https://i.ytimg.com/vi/IuO5K4ZVxrc/maxresdefau...,1.0,NaN,NaN,NaN,23446.0,783.0,NaN,0.0,150.0
1722,1722,Watched Douglas Exclusive Beauty Highlights 20...,https://www.youtube.com/watch?v=VBkpWfWOYQc,2025-09-04T09:35:14.188Z,group02,4,VBkpWfWOYQc,1390,UCnYwJm9kVAs2p-88g1UjnCA,Helga Mua,...,https://i.ytimg.com/vi/VBkpWfWOYQc/maxresdefau...,1.0,Munich,48.135125,11.581981,5419.0,147.0,NaN,0.0,41.0
1723,1723,Watched HOURGLASS AMBIENT LIGHTING EDIT UNLOCK...,https://www.youtube.com/watch?v=2vzgW6sOS-I,2025-09-04T09:32:26.458Z,group02,4,2vzgW6sOS-I,1391,UChK6kpyOTrMOJ391JUYBfSw,Charlotte Holdcroft,...,https://i.ytimg.com/vi/2vzgW6sOS-I/maxresdefau...,1.0,NaN,NaN,NaN,27466.0,2389.0,NaN,0.0,326.0


### Exporting the datasets

After completing the work, you may want to export the appropriate datasets. The example below has the dataset_participant_level as an example.

In [12]:
watch_history_details.to_excel('watch_history_details.xlsx', index=False)